In [1]:
import itertools
import numpy as  np

In [2]:
N = 10
M = 3

towers = np.array(['a', 'b', 'c'])
towers_index = {
    'a': 0,
    'b': 1,
    'c': 2
}

## Towers users per time slot

Specifies for each tower the number of users a the moments t0, t1 and t2

In [3]:
P = np.array([
    [3, 2, 5],  # P_t0
    [1, 3, 6],  # P_t1
    [4, 2, 4],  # P_t2
])

## Building distribution matrix

Generates the users distribution matrix for each defined time period in P

In [4]:
L = []

for Pt in P:
    L.append(
        np.array(list(
            itertools.chain(*(
                [towers[tower_index]] * count
                for tower_index, count in enumerate(Pt)
            ))
        )
    ))

L

[array(['a', 'a', 'a', 'b', 'b', 'c', 'c', 'c', 'c', 'c'], dtype='<U1'),
 array(['a', 'b', 'b', 'b', 'c', 'c', 'c', 'c', 'c', 'c'], dtype='<U1'),
 array(['a', 'a', 'a', 'a', 'b', 'b', 'c', 'c', 'c', 'c'], dtype='<U1')]

## Build random trajectories

Generates the recovered trajectory for the users like a permutation of the users distribution (L) until the time t1

In [5]:
S_t1 = np.array([
    np.random.permutation(Li)
    for Li in L[:2]
]).T

S_t1

array([['c', 'b'],
       ['c', 'c'],
       ['b', 'b'],
       ['a', 'c'],
       ['c', 'b'],
       ['c', 'a'],
       ['a', 'c'],
       ['b', 'c'],
       ['a', 'c'],
       ['c', 'c']], dtype='<U1')

# Night scenario

## Calculate possible next stops for each user

Calculate next estimate as the last tower the users were until time t1

In [6]:
L_t2_est = S_t1[:,1]

L_t2_est

array(['b', 'c', 'b', 'c', 'b', 'a', 'c', 'c', 'c', 'c'], dtype='<U1')

Night probability of users switching between towers

In [7]:
# CHANGING PROBABILITIES TO RAW DISTANCES
#prob_night = np.array([
#     [0.88, 0.04, 0.08],
#     [0.06, 0.90, 0.04],
#     [0.01, 0.09, 0.9],
# ])

# prob_night
prob_night = np.array([
    [0, 4, 3],
    [4, 0, 5],
    [3, 5, 0],
])

Convert tower letters into numbers to later access the prob_night matrix easier

In [8]:
L_t2_est_index = [towers_index[t] for t in L_t2_est]
L_t2_index = [towers_index[t] for t in L[2]]

Estimated tower index for users at time t2 and users distribution for time t2

In [9]:
list(zip(L_t2_est_index, L_t2_index))

[(1, 0),
 (2, 0),
 (1, 0),
 (2, 0),
 (1, 1),
 (0, 1),
 (2, 2),
 (2, 2),
 (2, 2),
 (2, 2)]

## Build cost matrix

Calculates the cost matrix at t1 as the probability of jumping between towers at night, taking into account the estimated towers for the time t2

In [10]:
C_t1 = np.zeros((N, N))

In [11]:
for i, l_t2_est in enumerate(L_t2_est_index):
    for j, l_t2 in enumerate(L_t2_index):
        C_t1[i, j] = prob_night[l_t2_est, l_t2]

In [12]:
C_t1

array([[4., 4., 4., 4., 0., 0., 5., 5., 5., 5.],
       [3., 3., 3., 3., 5., 5., 0., 0., 0., 0.],
       [4., 4., 4., 4., 0., 0., 5., 5., 5., 5.],
       [3., 3., 3., 3., 5., 5., 0., 0., 0., 0.],
       [4., 4., 4., 4., 0., 0., 5., 5., 5., 5.],
       [0., 0., 0., 0., 4., 4., 3., 3., 3., 3.],
       [3., 3., 3., 3., 5., 5., 0., 0., 0., 0.],
       [3., 3., 3., 3., 5., 5., 0., 0., 0., 0.],
       [3., 3., 3., 3., 5., 5., 0., 0., 0., 0.],
       [3., 3., 3., 3., 5., 5., 0., 0., 0., 0.]])

In [13]:
from scipy.optimize import linear_sum_assignment

# Ref: https://docs.scipy.org/doc/scipy-0.18.1/reference/generated/scipy.optimize.linear_sum_assignment.html
row_ind, col_ind = linear_sum_assignment(C_t1)

list(zip(row_ind, col_ind))

[(0, 4),
 (1, 6),
 (2, 5),
 (3, 7),
 (4, 0),
 (5, 3),
 (6, 8),
 (7, 9),
 (8, 2),
 (9, 1)]

# Daylight scenario

Try building a cost matrix using the daylight probabilities specifications